# LASSO regression of FI vs. GO terms TF-IDF

## Preparation

In [1]:
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
from sklearn import linear_model
import matplotlib.pyplot as plt

### Import GO data

In [2]:
with open('../processed_data/geneOntologyDataset.pkl', 'rb') as inp:
    [pivoted, tfidfPivoted] = pkl.load(inp)

In [3]:
GOdata = pd.DataFrame(tfidfPivoted['P'].todense()) #.join(pd.DataFrame(tfidfPivoted['P'].todense()), lsuffix='_left', rsuffix='_right').join(pd.DataFrame(tfidfPivoted['C'].todense()), lsuffix='_left2', rsuffix='_right2') # F / P / C
GOdata["Standard Name"] = pivoted['P'].index

### Import FI data & merge

In [4]:
FIdata = pd.read_excel('../raw_data/41592_2018_45_MOESM4_ESM.xlsx', skiprows = 6, engine = 'openpyxl')
full_data = pd.merge(GOdata, FIdata, on = 'Standard Name', how = 'left')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [45]:
full_data.index = full_data["Systematic Name"]
full_data
x = full_data.iloc[:, 0:full_data.columns.get_loc('Standard Name')]
x.columns = pivoted.get("P").columns
y = full_data["intensity"]

In [49]:
mask = ~y.isnull() & (y > 0)
whole_y = np.log(y[mask])
vmask = whole_y > -20
y = whole_y[vmask]

In [50]:
xy = pd.merge(x, y, how = "right", left_index = True, right_index = True)

## LASSO

In [58]:
clf = linear_model.LassoLarsIC(criterion = 'bic', normalize = False, eps = 1e-)

In [ ]:
clf.fit(xy.iloc[:,0:-1], xy.iloc[:,-1])

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_least_angle.py:651: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 159 iterations, i.e. alpha=1.347e-03, with an active set of 159 regressors, and the smallest cholesky pivot element being 1.490e-08. Reduce max_iter or increase eps parameters.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_least_angle.py:651: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 393 iterations, i.e. alpha=6.517e-04, with an active set of 389 regressors, and the smallest cholesky pivot element being 1.054e-08. Reduce max_iter or increase eps parameters.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_least_angle.py:651: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, aft

In [10]:
# number of GO terms with nonzero coefficient
np.sum([clf.coef_ != 0])

59

In [11]:
res = pd.DataFrame(clf.coef_)
res.index = x.columns

In [17]:
selection = res[res[0] != 0]
selection = selection.sort_values(by = 0)

In [13]:
# R^2 of the model
clf.score(xy.iloc[:,0:-1], xy.iloc[:,-1])

0.511620664265426

In [18]:
selection.to_excel("lasso.xlsx")